In [72]:
import os                 # Operation system related commands

## Data manipulation
import os #----to save files on operating system
import numpy as np
import pandas as pd
import datetime as dt
import pandas_profiling  #conda install -c anaconda pandas-profiling
import re as re

import statsmodels.formula.api as smf
import scipy.stats as stats
import pandas_profiling   #need to install using anaconda prompt (pip install pandas_profiling)


## Visualization
import scipy.stats as stats
import matplotlib.pyplot as plt
%pylab inline  
import seaborn as sns
# To show visulization output in Jupyter notebooks
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True
plt.gray()

from matplotlib.backends.backend_pdf import PdfPages



# Before model building
#from sklearn.model_selection import train_test_split
#import sklearn.metrics as metrics
#import sklearn.preprocessing as preprocess
#import sklearn.feature_selection as feature_selection

#Model Building Linear Regression
#import statsmodels as st
#import statsmodels.formula.api as smp
#from sklearn.linear_model import LinearRegression
#Selectively import to ensure that the hige package does not load your system


# Model Building for logistic Regression
#from sklearn.model_selection import train_test_split
#from sklearn import metrics
#from sklearn.linear_model import LogisticRegression
#from statsmodels.stats.outliers_influence import variance_inflation_factor

Populating the interactive namespace from numpy and matplotlib


<Figure size 720x540 with 0 Axes>

In [73]:
%pwd
rfm_retail=pd.read_csv("DATA SET FOR RFM-VALUE BASED SEGMENTATION.CSV")
rfm_retail.info()

# No missing
# No chnage in column name required

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22602 entries, 0 to 22601
Data columns (total 9 columns):
cust_id                 22602 non-null int64
No_of_days_frm_L_pur    22602 non-null int64
total_sale              22602 non-null float64
No_baskets              22602 non-null int64
item_cnt                22602 non-null int64
no_SKUs                 22602 non-null int64
no_Categories           22602 non-null int64
avg_no_prds_w_ret       22602 non-null float64
Flag_Multi_cat_p2       22602 non-null int64
dtypes: float64(2), int64(7)
memory usage: 1.6 MB


In [74]:
#rfm_retail.drop(['cust_id','no_Categories'], axis=1, inplace=True)
rfm_retail.rename(columns={'total_sale':'Monetary_value',
                           'No_baskets': 'frequency',
                           'No_of_days_frm_L_pur': 'recency'  }, inplace=True)
rfm_retail

,cust_id,recency,Monetary_value,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2
0,347,3,444.00,4,47,25,5,11.75000,0
1,585,320,509871.48,27,19322,531,68,715.62963,0
2,885,115,2284.59,4,247,130,17,61.75000,1
3,1670,3,8092.17,28,553,306,114,19.75000,1
4,2188,428,200.50,1,51,14,2,51.00000,0
...,...,...,...,...,...,...,...,...,...
22597,132473524,0,108.90,1,11,4,3,11.00000,1
22598,132484207,0,101.86,1,7,2,1,7.00000,0
22599,132489383,0,169.00,1,15,1,1,15.00000,0
22600,132490575,0,62.80,1,3,3,1,3.00000,0


In [82]:
quantiles=rfm_retail[['frequency','Monetary_value','recency']].quantile(q=[0.25,0.5,0.75])
print(quantiles)
quantiles=quantiles.to_dict()
quantiles

      frequency  Monetary_value  recency
0.25        1.0         151.800     26.0
0.50        2.0         393.755     93.0
0.75        5.0        1117.725    225.0


{'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'Monetary_value': {0.25: 151.8, 0.5: 393.755, 0.75: 1117.725},
 'recency': {0.25: 26.0, 0.5: 93.0, 0.75: 225.0}}

In [81]:
# High recency is bad; High frequency and monetary is good
# Arguments (x=value,  p= val of recency., freq, monetary,  d= quartile decile)
def Rclass(x,p,d):
    if x<d[p][0.25]:
        return 1
    elif x<d[p][0.5]:
        return 2
    elif x<d[p][0.75]:
        return 3
    else:
        return 4
    
def FMclass(x,p,d):
    if xd[p][0.25]:
        return 4
    elif x<d[p][0.5]:
        return 3
    elif x<d[p][0.75]:
        return 2
    else:
        return 1

In [133]:
rfm_retail['R_Quartile']=rfm_retail['recency'].apply(Rclass, args=('recency',quantiles,))
rfm_retail['F_Quartile']=rfm_retail['frequency'].apply(FMclass, args=('frequency',quantiles,))
rfm_retail['M_Quartile']=rfm_retail['Monetary_value'].apply(FMclass, args=('Monetary_value',quantiles,))



In [134]:
rfm_retail1=rfm_retail.assign(R_seg=np.nan, FM_seg=np.nan)
rfm_retail1.R_seg[rfm_retail["R_Quartile"]==1]="1_Active"
rfm_retail1.R_seg[rfm_retail["R_Quartile"]==2]="2_At_Risk"
rfm_retail1.R_seg[rfm_retail["R_Quartile"]==3]="2_At_Risk"
rfm_retail1.R_seg[rfm_retail["R_Quartile"]==4]="4_Churn"

rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==1) & (rfm_retail["M_Quartile"]==1)] = "1_Premium"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==1) & (rfm_retail["M_Quartile"]==2)] = "2_Gold"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==1) & (rfm_retail["M_Quartile"]==3)] = "2_Gold"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==1) & (rfm_retail["M_Quartile"]==4)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==2) & (rfm_retail["M_Quartile"]==1)] = "2_Gold"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==2) & (rfm_retail["M_Quartile"]==2)] = "2_Gold"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==2) & (rfm_retail["M_Quartile"]==3)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==2) & (rfm_retail["M_Quartile"]==4)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==3) & (rfm_retail["M_Quartile"]==1)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==3) & (rfm_retail["M_Quartile"]==2)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==3) & (rfm_retail["M_Quartile"]==3)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==3) & (rfm_retail["M_Quartile"]==4)] = "4_Standard"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==4) & (rfm_retail["M_Quartile"]==1)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==4) & (rfm_retail["M_Quartile"]==2)] = "3_Silver"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==4) & (rfm_retail["M_Quartile"]==3)] = "4_Standard"
rfm_retail1.FM_seg[(rfm_retail["F_Quartile"]==4) & (rfm_retail["M_Quartile"]==4)] = "4_Standard"



#freq\Mon      # 1      2          3            4
# 1         Premium     Gold      Gold         Silver
# 2           Gold      Gold      Silver       Silver
# 3         Silver      Silver    Silver       Standard
# 4         Silver     Silver     Standard     Standard

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the 

In [135]:
rfm_retail1.head(3)

,cust_id,recency,Monetary_value,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,R_Quartile,F_Quartile,M_Quartile,R_seg,FM_seg
0,347,3,444.00,4,47,25,5,11.75000,0,1,2,2,1_Active,2_Gold
1,585,320,509871.48,27,19322,531,68,715.62963,0,4,1,1,4_Churn,1_Premium
2,885,115,2284.59,4,247,130,17,61.75000,1,3,2,1,2_At_Risk,2_Gold


In [136]:
#Profiling
col_kpi=['recency', 'Monetary_value','frequency','item_cnt','no_SKUs','no_Categories','avg_no_prds_w_ret','Flag_Multi_cat_p2']
VB_Profiling=rfm_retail1.groupby(['R_seg','FM_seg']).apply(lambda x: x[col_kpi].mean()).T

In [137]:
VB_Profiling.to_csv('full_rfm_retail_profiling_output.csv')

In [138]:
VB_Profiling

R_seg                 1_Active                                      \
FM_seg               1_Premium      2_Gold    3_Silver  4_Standard   
recency              10.331337   11.467890   11.789054   11.414737   
Monetary_value     4564.687061  887.446101  425.120730  101.925368   
frequency            16.867840    4.503277    1.859749    1.000000   
item_cnt            277.179334   55.175623   21.507412    8.212632   
no_SKUs             119.840509   27.687418   10.400228    4.494737   
no_Categories        45.023961   10.648755    3.992018    1.898947   
avg_no_prds_w_ret    16.067401   12.874009   13.108704    8.212632   
Flag_Multi_cat_p2     0.986896    0.932503    0.737742    0.532632   

R_seg                2_At_Risk                                       \
FM_seg               1_Premium       2_Gold    3_Silver  4_Standard   
recency              72.557106    95.027306  110.546244  120.811072   
Monetary_value     3111.580377  1008.688853  388.679683   98.649307   
frequency            11.379845     4.064927    1.649944    1.000000   
item_cnt            188.982429    56.443871   20.194507    8.041425   
no_SKUs              78.270801    25.107706    8.941704    4.334495   
no_Categories        29.000000     9.366505    3.400504    1.816105   
avg_no_prds_w_ret    16.477911    14.718984   13.586977    8.041425   
Flag_Multi_cat_p2     0.901292     0.772451    0.545123    0.422764   

R_seg                  4_Churn                                       
FM_seg               1_Premium       2_Gold    3_Silver  4_Standard  
recency             281.987013   308.519846  334.025884  350.765095  
Monetary_value     8259.388701  1235.298643  496.328154   94.633693  
frequency             8.902597     3.443022    1.835203    1.000000  
item_cnt            293.155844    51.201024   21.674288    8.588916  
no_SKUs              63.058442    19.509603    8.449957    4.592225  
no_Categories        21.266234     7.918054    3.748059    2.092225  
avg_no_prds_w_ret    23.165109    15.940928   13.235562    8.588916  
Flag_Multi_cat_p2     0.000000     0.000000    0.000000    0.000000